In [254]:
# include modules
import os
import pandas as pd
import itertools 
from collections import Counter 
import numpy as np

pd.set_option('display.max_rows', None)

In [255]:
# go to working directory
os.chdir('C:\\Users\\user\\Desktop\\research\\data')
os.getcwd()

'C:\\Users\\user\\Desktop\\research\\data'

In [256]:
#open each csv file
for file in os.listdir():
    if(os.path.splitext(file)[1] == '.csv'):
        filecsv = pd.read_csv(file)
        f_name,f_ext = os.path.splitext(file)

In [257]:
# display csv
filecsv

,Author,Commit,Message,Date
0,Hanif Ramadhan,2681e9b,Final,2019‑04‑29
1,dennis cunong,3d514ee,Rotate Landscape Feed,2019‑04‑26
2,Ilham Satria,5cbeff8,Rotate Landscape Home,2019‑04‑26
3,aprilia widyas,16ce644,intent setting,2019‑04‑26
4,Hanif Ramadhan,cf73c41,Merapihkan Kodingan,2019‑04‑26
5,Rizka Fadhilah,a359369,Perbaikan backend Transportasi#64,2019‑04‑25
6,Hanif Ramadhan,4c3e411,Search,2019‑04‑24
7,Hanif Ramadhan,1db2f07,Search,2019‑04‑24
8,Ilham Satria,175e3b5,Back end fitur Search #56,2019‑04‑22
9,Rizka Fadhilah,d81e095,Backend menu Transprtasi Resolved#64,2019‑04‑21


In [258]:
# activity list
membuat = ["buat", "create"]
menambahkan = ["ambah", "add"]
mengubah = ["ubah", "edit"]
update = ["update"]
menghubungkan = ["onek", "connect", "ubung", "ambung"]
memperbaiki = ["baik", "revisi", "fix", "betulkan"]
melengkapi = ["lengkapi"]
testing = ["test"]
resolved = ["resolve"]
merge = ['merge']
kata_sambung = ["dan", "#", "+", ",", "dari", "dengan", "pada", "untuk", "ref", "&", "http"]
conditional = ["fungsional", "layout", "fitur", "menampilkan"]
allAct = membuat + menambahkan + mengubah + update + menghubungkan + memperbaiki + melengkapi + testing + resolved + kata_sambung + merge + conditional

In [259]:
#duplicate series and make it in lower case
aktivitas = filecsv['Message']
aktivitas = aktivitas.str.lower()

In [260]:
# make new activity column
for string in membuat:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'membuat'
for string in menambahkan:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'menambahkan'
for string in mengubah:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'mengubah'
for string in update:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'update'
for string in menghubungkan:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'menghubungkan'
for string in memperbaiki:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'memperbaiki'
for string in melengkapi:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'melengkapi'
for string in testing:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'testing'
for string in resolved:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'resolved'
for string in merge:
    filecsv.loc[aktivitas.str.contains(string), 'activity'] = 'merge'
#   file.activity = file.activity.replace(np.nan, 'menambahkan', regex=True)

In [261]:
# split series into list
aktivitas = aktivitas.str.split()

In [262]:
# delete activity keyword from series
for upItem in aktivitas:
    for items in upItem:
        for act in allAct:
            upItem[:] = ['qwerty' if act in items  else items for items in upItem]

for upItem in aktivitas:
    if 'qwerty' in upItem:
        upItem[:] = list(dict.fromkeys(upItem))
        upItem.remove('qwerty')

        

In [263]:
# convert fitur into string and store it in list
aktivitas2 = []
for items in aktivitas:
    items = " ".join(items)
    aktivitas2.append(items)
    
# make it short
shortAct = []
for items in aktivitas2:
    if (len(items.split())) < 3:
        shortAct.append(items)
    else:
        items = items.split()
        a = items[0] + ' ' + items[1]
        shortAct.append(a)

In [264]:
# create new frame
newFrame = pd.DataFrame(columns = ['Case ID', 'Timestamp', 'Activity', 'Resource'])  
newFrame['Timestamp'] = filecsv['Date']
newFrame['Activity'] = filecsv['activity']
newFrame['Resource'] = filecsv['Author']
newFrame.Activity = newFrame.Activity.replace(np.nan, 'menambahkan', regex=True)

In [265]:
# join activity with fitur
feature = newFrame['Activity']
feature = feature.str.split()
i = 0
for items in feature:
    items.append(shortAct[i])
    items = " ".join(items)
    i+=1

# convert into list
aktifitasbaru = []
for items in feature:
    items = " ".join(items)
    aktifitasbaru.append(items)

newFrame['Activity'] = aktifitasbaru

In [266]:
# grouping fitur
shortAct[:] = list(dict.fromkeys(shortAct))
shortAct.remove('')
shortAct

['final',
 'rotate landscape',
 'intent setting',
 'merapihkan kodingan',
 'backend',
 'search',
 'back end',
 'backend menu',
 'setting share',
 'front end',
 'menyelesaikan feed',
 'progress setting',
 'profile',
 'filter kategori',
 'tranportasi',
 'front-end about',
 'progress',
 "branch 'interface-improvement'",
 'workspace.xml',
 'solved backend',
 'solve error',
 'solve conflict',
 'branch',
 'remote-tracking branch',
 'in dennis_konten',
 'frontend backend',
 'final backend',
 'feed',
 'backend filter',
 'rating baru',
 'backend registrasi',
 'pencarian lokasi',
 'diskusi',
 'feedback seluruh',
 'login with',
 'upload konten',
 'sidebar 1',
 "branch 'master'",
 'rating pariwisata',
 'feedback',
 'foto ke',
 'upload foto',
 'navbar bottom',
 'bottom navigation',
 'solved',
 'login facebook',
 'coba commit',
 'sidebar',
 'front-end pariwisata',
 'splash screen',
 'first commit',
 'readme.md online',
 'initial commit']

In [267]:
# grouping fitur into caseID
caseid = [0] * len(aktifitasbaru)

i = 1
aktifitasbaru
for items in shortAct:
    y = 0
    d = False
    for item in aktifitasbaru:
        if items in item:
            caseid[y] = i
            d = True
        y+=1
        
    if d:
        i+=1
    

In [268]:
# create new column
newFrame['Case ID'] = pd.Series(caseid)

In [269]:
# display new frame
newFrame

,Case ID,Timestamp,Activity,Resource
0,1,2019‑04‑29,menambahkan final,Hanif Ramadhan
1,2,2019‑04‑26,menambahkan rotate landscape,dennis cunong
2,2,2019‑04‑26,menambahkan rotate landscape,Ilham Satria
3,3,2019‑04‑26,menambahkan intent setting,aprilia widyas
4,4,2019‑04‑26,menambahkan merapihkan kodingan,Hanif Ramadhan
5,5,2019‑04‑25,memperbaiki backend,Rizka Fadhilah
6,6,2019‑04‑24,menambahkan search,Hanif Ramadhan
7,6,2019‑04‑24,menambahkan search,Hanif Ramadhan
8,7,2019‑04‑22,menambahkan back end,Ilham Satria
9,45,2019‑04‑21,resolved backend menu,Rizka Fadhilah


In [270]:
os.chdir('C:\\Users\\user\\Desktop\\research\\')
if os.path.isdir('./converted'):
    os.chdir('./converted')
    newfile = '{}{}'.format(f_name, f_ext)
    newFrame.to_csv(newfile)
    os.chdir('../data')
else:
    os.mkdir('converted')
    os.chdir('./converted')
    newfile = '{}{}'.format(f_name, f_ext)
    newFrame.to_csv(newfile)
    os.chdir('../data')

gaada
